In [1]:
import pandas as pd
import re
import cleaning_functions as cf

In [2]:
oscars = pd.read_csv('../data/Clean/1_oscars_cleaned.csv')
oscars.head(2)

,year_film,year_ceremony,Oscars nominations,name,film,Oscars winner
0,1967,1968,Actor in a leading role,Warren Beatty,Bonnie and Clyde,False
1,1967,1968,Actor in a leading role,Dustin Hoffman,The Graduate,False


In [3]:
oscars.replace('Star Wars', 'Star Wars: Episode IV - A New Hope', inplace = True)

In [4]:
golden = pd.read_csv('../data/Clean/2_golden_final.csv')
golden.head(2)

,year_film,year_award,Golden Globes nominations,nominee,film,Golden Globes winner
0,1967,1968,Best motion picture - drama,In the Heat of the Night,In the Heat of the Night,True
1,1967,1968,Best motion picture - drama,Bonnie and Clyde,Bonnie and Clyde,False


In [5]:
#renombramos algunas columnas para mantener los datos claros después de unirlas:
golden.rename(columns={"nominee": "G.G. name", 'Golden Globes nominations':'G.G. nominations', 'Golden Globes winner':'G.G. winner'}, inplace=True)

In [6]:
oscars.rename(columns={"name": "Oscars name"}, inplace=True)

In [7]:
golden.rename(columns={"year_award": "year_ceremony"}, inplace=True)

In [8]:
golden.head(1)

,year_film,year_ceremony,G.G. nominations,G.G. name,film,G.G. winner
0,1967,1968,Best motion picture - drama,In the Heat of the Night,In the Heat of the Night,True


In [9]:
oscars.head(1)

,year_film,year_ceremony,Oscars nominations,Oscars name,film,Oscars winner
0,1967,1968,Actor in a leading role,Warren Beatty,Bonnie and Clyde,False


In [10]:
oscars.reset_index(drop = True, inplace = True)

In [11]:
#cambiamos el formato de los df con un pivot table para poder tener las categorías como nombres de columna expandidos para cada película (unmelt)

In [12]:
oscars_films = oscars.pivot_table(index= ['film','year_film','year_ceremony'], columns = 'Oscars nominations', values= 'Oscars winner')
oscars_films = oscars_films.reset_index()
oscars_films.columns.name = None
oscars_films.head(3)

,film,year_film,year_ceremony,Actor in a leading role,Actor in a supporting role,Actress in a leading role,Actress in a supporting role,Adapted screenplay,Animated feature film,Best picture,...,International feature film,Makeup and hairstyling,Original score,Original screenplay,Original song,Production design,Short film animated,Short film live action,Sound effects,Visual effects
0,'38',1986,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,'Crocodile' Dundee,1986,1987,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,'Round Midnight,1986,1987,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
golden_films = golden.pivot_table(index= ['film','year_film','year_ceremony'], columns = 'G.G. nominations', values= 'G.G. winner')
golden_films = golden_films.reset_index()
golden_films.columns.name = None
golden_films.head(2)

,film,year_film,year_ceremony,Best actor - drama,Best actor - musical or comedy,Best actress - drama,Best actress - musical or comedy,Best director - motion picture,Best motion picture - animated,Best motion picture - drama,Best motion picture - foreign language,Best motion picture - musical or comedy,Best original score - motion picture,Best original song - motion picture,Best screenplay - motion picture,Best supporting actor - motion picture,Best supporting actress - motion picture,Documentary
0,Roma (1972),1972,1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"$1,000,000 Duck",1971,1972,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#creamos tres columnas nuevas por cada df: 
    #número de nominaciones (sumar columnas con valor distinto de NaN)
    #número de premios (sumar fila)
    #y si han recibido el premio a la mejor película - posteriormente cambiaremos NaN por 0

In [15]:
oscars_total_nominations = []
total_oscars_won = []
oscars_best_picture = []
for n in range(len(oscars_films)):
    oscars_total_nominations.append(int(oscars_films.iloc[n,3:].notnull().sum()))
    total_oscars_won.append(int(oscars_films.iloc[n,3:].sum()))
    if oscars_films.loc[n,'Best picture'] == True:
        oscars_best_picture.append(1)
    else:
        oscars_best_picture.append(0)

In [16]:
oscars_films['Total oscars nominations'] = oscars_total_nominations
oscars_films['Total oscars won'] = total_oscars_won
oscars_films['Oscars best picture'] = oscars_best_picture
oscars_films.head(3)

,film,year_film,year_ceremony,Actor in a leading role,Actor in a supporting role,Actress in a leading role,Actress in a supporting role,Adapted screenplay,Animated feature film,Best picture,...,Original screenplay,Original song,Production design,Short film animated,Short film live action,Sound effects,Visual effects,Total oscars nominations,Total oscars won,Oscars best picture
0,'38',1986,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
1,'Crocodile' Dundee,1986,1987,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
2,'Round Midnight,1986,1987,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0


In [17]:
golden_total_nominations = []
total_golden_won = []
golden_best_picture = []
for n in range(len(golden_films)):
    golden_total_nominations.append(int(golden_films.iloc[n,3:].notnull().sum()))
    total_golden_won.append(int(golden_films.iloc[n,3:].sum()))
    if golden_films.loc[n,'Best motion picture - drama'] == True:
        golden_best_picture.append(1)
    elif golden_films.loc[n,'Best motion picture - musical or comedy'] == True:
        golden_best_picture.append(1)
    else:
        golden_best_picture.append(0)

In [19]:
golden_films['G.G. nominations'] = golden_total_nominations
golden_films['G.G. won'] = total_golden_won
golden_films['G.G. best motion picture'] = golden_best_picture
golden_films.head(2)

,film,year_film,year_ceremony,Best actor - drama,Best actor - musical or comedy,Best actress - drama,Best actress - musical or comedy,Best director - motion picture,Best motion picture - animated,Best motion picture - drama,...,Best motion picture - musical or comedy,Best original score - motion picture,Best original song - motion picture,Best screenplay - motion picture,Best supporting actor - motion picture,Best supporting actress - motion picture,Documentary,G.G. nominations,G.G. won,G.G. best motion picture
0,Roma (1972),1972,1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
1,"$1,000,000 Duck",1971,1972,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0


In [20]:
#creamos un subset de cada df solo con las columnas que nos interesan para juntarlas en un último dataframe

In [21]:
subset_oscars = oscars_films[['film','year_film','Total oscars nominations','Total oscars won','Oscars best picture']]
subset_golden = golden_films[['film','year_film', 'G.G. nominations', 'G.G. won', 'G.G. best motion picture']]

In [31]:
award_winning_movies = pd.merge(subset_oscars,subset_golden, how='outer', on = ['film','year_film'])
award_winning_movies = award_winning_movies.sort_values(by='year_film')
award_winning_movies.reset_index(drop = True, inplace = True)
award_winning_movies.head(60)

,film,year_film,Total oscars nominations,Total oscars won,Oscars best picture,G.G. nominations,G.G. won,G.G. best motion picture
0,Far from the Madding Crowd,1967,1.0,0.0,0.0,NaN,NaN,NaN
1,I Even Met Happy Gypsies,1967,1.0,0.0,0.0,NaN,NaN,NaN
2,Ski Fever,1967,NaN,NaN,NaN,1.0,0.0,0.0
3,El Amor Brujo,1967,1.0,0.0,0.0,NaN,NaN,NaN
4,A King's Story,1967,1.0,0.0,0.0,NaN,NaN,NaN
5,In Cold Blood,1967,4.0,0.0,0.0,1.0,0.0,0.0
6,In the Heat of the Night,1967,6.0,4.0,1.0,5.0,2.0,1.0
7,Smashing Time,1967,NaN,NaN,NaN,1.0,0.0,0.0
8,The Stranger,1967,NaN,NaN,NaN,1.0,0.0,0.0
9,The Comedians,1967,NaN,NaN,NaN,1.0,0.0,0.0


In [32]:
#award_winning_movies.to_csv('3_award_winning_movies.csv')
